In [1]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,GlobalAveragePooling2D,Flatten
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
bottleneck_train=np.load('bottleneck_train_vgg16_cancdetect')
bottleneck_valid=np.load('bottleneck_valid_vgg16_cancdetect')
bottleneck_test=np.load('bottleneck_test_vgg16_cancdetect')

In [ ]:
bottleneck_train.shape

(2000, 7, 7, 512)

In [ ]:
trainfiles=load_files('data/train/')
train_filenames=trainfiles['filenames']
train_targets=np_utils.to_categorical(trainfiles['target'])

validfiles=load_files('data/valid/')
valid_filenames=validfiles['filenames']
valid_targets=np_utils.to_categorical(validfiles['target'])

testfiles=load_files('data/test/')
test_filenames=testfiles['filenames']
test_targets=np_utils.to_categorical(testfiles['target'])

In [ ]:
model=Sequential()
model.add(GlobalAveragePooling2D(input_shape=bottleneck_train.shape[1:]))
# model.add(Dense(units=1000,activation='relu'))
model.add(Dense(units=len(trainfiles['target_names']),activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='rmsprop')

In [ ]:

checkpointer=ModelCheckpoint(filepath='cancdetect_transfer_vgg16.weights.best.hdf5',verbose=1,save_best_only=True)

model.fit(bottleneck_train,train_targets,batch_size=10,epochs=100,shuffle=True,verbose=1,callbacks=[checkpointer],
             validation_data=[bottleneck_valid,valid_targets])

In [ ]:
model.load_weights('cancdetect_transfer_vgg16.weights.best.hdf5')
predictions=model.predict(bottleneck_test)

In [ ]:

ids=[filename for filename in test_filenames]
ids=[x.replace('\\','/') for x in ids]
sub_csv=[(x,y,z) for x,y,z in zip(ids,predictions[:,0],predictions[:,2])]

In [ ]:
sub_csv=[('Id','task_1','task_2')]+sub_csv

In [ ]:
import csv
with open('results_vgg16.csv', 'w',newline='')  as myfile:
    writer = csv.writer(myfile)
#     writer.writerow('file,species')
    writer.writerows(sub_csv)